In [ ]:
using Knet, Plots, Statistics
# using IterTools, Base.Iterators # take, cycle, takenth
# using StatsBase, Statistics # mean, describe
ENV["COLUMNS"]=40
ARRAY = KnetArray{Float64}

In [ ]:
# Load mnist data
include(Knet.dir("data/mnist.jl"))
xtrn,ytrn,xtst,ytst = mnist()
println.(summary.((xtrn,ytrn,xtst,ytst)));

In [ ]:
# Minibatched reshaped version
dtrn = minibatch(xtrn,ytrn,100;xtype=ARRAY,xsize=(784,:),shuffle=true)
x1,y1 = first(dtrn)
println.(summary.((dtrn,x1,y1)));

In [ ]:
# Train linear model until convergence
w = param(10,784,atype=ARRAY)
pred(w,x) = w * x
loss(w,x,y) = nll(pred(w,x),y)
losses = collect(progress(sgd((x,y)->loss(w,x,y), repeat(dtrn,20))));

In [ ]:
plot(losses)

In [ ]:
∇loss(w,x,y) = grad((@diff loss(w,x,y)), w)

function diffusiontensor(w,data,lr=0.1)
    grads = [ ∇loss(w,x,y) for (x,y) in data ]
    fullgrad = mean(grads)
    errors = [ fullgrad - g for g in grads ]
    erravg = mean(errors)
    errvec = vec(erravg)
    dt = errvec * errvec'
end

In [ ]:
dt = diffusiontensor(w,dtrn)
extrema(Array(dt))